In [6]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

In [7]:
def hod_rd_sub1(mean):
    for i in range(my,shape[1]-lmd,lmd-2):
        ver_rd(i,mean)

In [8]:
def hod_rd_sub2(mean1):
    for i in range(my+lmd,0, -(lmd-2)):
        ver_rd(i,mean1)

In [9]:
def hor_rd(mean):
    hod_rd_sub1(mean)
    hod_rd_sub2(mean)

In [10]:
def ver_rd(x,mean):
    lm1 = lim1
    lm2 = lim2
    for i in range(mx,lmd+1,-(lmd-10)):
        lm1 += lm1*0.03
        lm2 += lm2*0.03
        low = mean - [0,lm1,lm2]
        high = mean + [180,lm1,lm2]
        newscr[i-lmd:i, x:x+lmd] = cv2.inRange(hls[i-lmd:i, x:x+lmd],low,high)
        res = [0]+[np.mean(hls[i-lmd:i, x:x+lmd,j]) for j in range(1,3)]
        res = np.array(res)
        if(abs(res[1]-mean[1])<=15 and abs(res[2]-mean[2])<=15):
            mean = (4*res+6*mean)/10

In [30]:
import numpy as np
import pandas as pd
import time
import cv2
import threading
# lim = 4*std
lmd = 40
lim1 = lim2 = 20
kernel = np.ones((3,3),np.uint8)

shape = [480,680]#scr.shape
scr2 = 0
mx = shape[0]
my = shape[1]//2
print("running")
kkk = 0 #300*8#*6
mask_prev1 = np.zeros((shape[0],shape[1]),np.uint8)
mask_prev2 = np.zeros((shape[0],shape[1]),np.uint8)
mean_prev = 0
vrec = cv2.VideoCapture('v5.mp4',0)

# while(kkk):
#     kkk-=1
#     ret, scr = vrec.read()

while True:
    start_time = time.time()
    scr = cv2.imread("dist.png",1)
#     if not ret:
#         print("not running")
#         break
    ret, scr = vrec.read()
    if not ret:
            break
    scr = cv2.resize(scr, (680,480)) 
    scr1 = scr
    scr2 = scr
    newscr = np.zeros((shape[0],shape[1]),np.uint8)
#     scr = cv2.bilateralFilter(scr,7,12,12)
    hls = cv2.cvtColor(scr, cv2.COLOR_BGR2HLS)
    mean = [0]+[np.mean(hls[440:480,200:480, i]) for i in range(1,3)]
    std = [np.std(hls[440:480,200:480, i]) for i in range(1,3)]
    lim1 = (lim1 + 16*std[0])/5  #(1:4(4))/5
    lim2 = (lim2 + 16*std[1])/5
#     print(lim1,lim2)
    mean = np.array(mean)
    mean = 0.6*mean + mean_prev*0.4
    mean_prev = mean
#     print(mean)
    hor_rd(mean)
    band =  cv2.bitwise_and(scr,scr,mask = newscr)
    
    gray = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
    mean1 = np.mean(gray[440:480,200:480])
    mean1 = np.array(mean1)
    low1 = mean1 - 25
    high1 = mean1 + 25
    
    mask1 = cv2.inRange(gray,low1,high1)
    band1 =  cv2.bitwise_and(scr,scr,mask = mask1)
#     newscr = cv2.morphologyEx(newscr,cv2.MORPH_OPEN,kernel, iterations = 1)
#     newscr = cv2.morphologyEx(newscr,cv2.MORPH_CLOSE,kernel, iterations = 1)
    
        
    mask3 = newscr+mask1
#     band3 =  cv2.bitwise_and(scr,scr,mask = mask3)
    
    mask4 = mask_prev1 + mask3 
#     mask_pres2 = mask_prev1
    mask_prev1 = mask3 
    
#     canny = cv2.Canny(gray,10,50)
#     cv2.imshow("canny", canny)

#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_OPEN,kernel, iterations = 1)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 1)
    
    
#     cv2.imshow('blur',band)
#     cv2.imshow('mean1',band1)
#     cv2.imshow('band3',band3)
#     cv2.imshow('newscr',newscr)
#     cv2.imshow('mean1',mask1)
    



    mask4 = cv2.morphologyEx(mask4,cv2.MORPH_OPEN,kernel, iterations = 1)
    mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 1)
    cv2.imshow('band3',mask4)
    dist_transform = cv2.distanceTransform(mask4,cv2.DIST_L2,5)
    ret, mask4 = cv2.threshold(dist_transform,0.07*dist_transform.max(),255,0)

    cv2.imshow('band4',mask4)
    mask4 = np.array(mask4)
    
    
    
#     mask4 = np.uint8(mask4)
#     im2, contours, hierarchy = cv2.findContours(mask4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
#     im3, newcontours, newhierarchy = cv2.findContours(mask4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     scr1 = cv2.drawContours(scr1, newcontours, -1, (0,255,255), 3)
#     cv2.imshow('scr1',scr1)
    
#     ncont=len(contours)
#     for i in range(ncont):
#         if(True):#len(contours[i])>100):
#             epsilon = 0.003*cv2.arcLength(contours[i], True)
#             contours[i] = cv2.approxPolyDP(contours[i], epsilon, True)
        

        
        
# #     cnt = contours[-5:]
# #     scr1 = cv2.drawContours(scr, contours, 3, (0,255,0), 3)
    
#     scr2 = cv2.drawContours(scr2, contours, -1, (255,0,0), 3)    
# #     scr1 = cv2.drawContours(scr, contours, 2, (0,0,255), 3)
# #     scr1 = cv2.drawContours(scr, contours, 4, (0,255,255), 3)
# #     scr1 = cv2.drawContours(scr, contours, 5, (255,0,255), 3)
# #     scr1 = cv2.drawContours(scr, contours, 6, (255,255,0), 3)
# #     scr1 = cv2.drawContours(scr, contours, 7, (255,255,255), 3)
# #     scr1 = cv2.drawContours(scr, contours, 3, (0,255,0), 3)
# #     scr1 = cv2.drawContours(scr, contours, 4, (0,255,0), 3)
# #     scr1 = cv2.drawContours(scr, contours, 5, (0,255,0), 3)
    
    
#     cv2.line(scr2,(0,480),(340,240),(255,255,0),2)
#     cv2.line(scr2,(680,480),(340,240),(255,255,0),2)
#     cv2.imshow('scr',scr2)
#     cv2.imshow('band4',mask4)
#     cv2.imshow('scr2',scr2)
#     print(mask4.shape)

    cv2.line(scr2,(0,480),(340,int(220)),(0,255,255),1)
    cv2.line(scr2,(680,480),(340,int(220)),(0,255,255),1)
    
    cv2.line(scr2,(0,int(220)),(680,int(220)),(0,0,255),2)#216  220
    
    for ind in range(0,260+20,20):
        xx = (ind)*2
        cv2.putText(scr2,str(xx),(10,480-ind), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,0),1,cv2.LINE_AA)
        cv2.line(scr2,(0,480-ind),(680,480-ind),(0,255,255),1)

    
    cv2.imshow('scr2',scr2)
    time.sleep(0.2)
#     print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running


In [12]:
m = 45

In [78]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

vrec = cv2.VideoCapture('v2.mp4',0)
shape = [480,680]

mx = shape[0]
my = shape[1]//2
print("running")
kkk = 300*6*6
while(kkk):
    kkk-=1
    ret, scr = vrec.read()

ret, scr1 = vrec.read()
scr1 = cv2.resize(scr1, (680,480)) 
scr1 = cv2.cvtColor(scr1, cv2.COLOR_BGR2GRAY)
ret, scr2 = vrec.read()
scr2 = cv2.resize(scr2, (680,480)) 
scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
while True:
    start_time = time.time()
    scr = scr1
    scr1 = scr2
    ret, scr2 = vrec.read()
#     ret, scr3 = vrec.read()
#     scr = np.mean(scr1,scr2,scr3)
    if not ret:
            break
    scr2 = cv2.resize(scr2, (680,480)) 
    scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
    
    d0 = abs(scr1-scr)
    d1 = abs(scr2-scr)
    d2=d0+d1
    
    cv2.imshow('d0',d0)
    cv2.imshow('d1',d1)
    cv2.imshow('d2',d2)
#     cv2.imshow('d3',d3)
#     cv2.imshow('d4',d4)

    cv2.imshow('scr1',scr)
    print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running
FPS:  6.805267291005497
FPS:  237.974695035461
FPS:  213.0818939239992
FPS:  233.77014825548991
FPS:  230.83676389653274
FPS:  211.23609991941981
FPS:  220.8458298230834
FPS:  214.39983642590605
FPS:  202.9174649250121
FPS:  214.3669630992538
FPS:  199.08410860072146
FPS:  201.45552353506244
FPS:  143.248087431694
FPS:  175.0471182337966
FPS:  165.47536197577622
FPS:  182.2421898761677
FPS:  90.95708368572853
FPS:  185.3590242177833
FPS:  185.9754356404913
FPS:  168.23649271990695
FPS:  185.3180753766624
FPS:  205.09041122683487
FPS:  223.36265843007774
FPS:  225.17335051269663
FPS:  155.42518342844437
FPS:  178.0945182794786
FPS:  232.61627197604128
FPS:  183.5260348297891
FPS:  243.2608746085141
FPS:  241.13510405887087
FPS:  225.84018953262978
FPS:  171.67958740943882
FPS:  217.8180307436643
FPS:  209.98818464003205
FPS:  222.87602954460917
FPS:  169.02974127508665
FPS:  147.40648063541155
FPS:  184.47853624208304
FPS:  212.76842692639374
FPS:  215.47927048548678
FPS:  157.6